In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
# Function to load and preprocess image using Keras
def load_and_preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
    return img_array

# Function to extract image paths from a folder
def extract_image_paths(root_folder, folder):
    image_paths = []
    for subdir, _, files in os.walk(os.path.join(root_folder, folder)):
        for file in files:
            if file.lower().endswith('.jpg'):
                image_path = os.path.join(subdir, file)
                image_paths.append(image_path)
    return image_paths

# Replace these paths with your actual dataset folder and subfolders
root_folder_path = "Reduced_Dataset"
real_folder = "Real"
fake_folder = "Fake"

In [ ]:
# Extract image paths for real and fake images
real_image_paths = extract_image_paths(root_folder_path, real_folder)
fake_image_paths = extract_image_paths(root_folder_path, fake_folder)

In [ ]:
# Create labels (0 for real, 1 for fake)
real_labels = np.full(len(real_image_paths), "Real")
fake_labels = np.full(len(fake_image_paths), "Fake")

In [ ]:
# Combine real and fake data
X = np.concatenate([real_image_paths, fake_image_paths], axis=0)
y = np.concatenate([real_labels, fake_labels], axis=0)
df = pd.DataFrame({'Image_Path': X, 'Nature': y})
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
print(df.tail())

In [ ]:
target_size = (224, 224)

# Create an ImageDataGenerator for real and fake images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)

# Create a flow_from_dataframe generator for training
train_generator = train_datagen.flow_from_dataframe(
    df,
    x_col='Image_Path',
    y_col='Nature',
    target_size=target_size,
    batch_size=8,
    class_mode='binary',
    subset='training'
)

# Create a flow_from_dataframe generator for validation
validation_generator = train_datagen.flow_from_dataframe(
    df,
    x_col='Image_Path',
    y_col='Nature',
    target_size=target_size,
    batch_size=8,
    class_mode='binary',
    subset='validation'
)



CheckPoint added

In [ ]:
# # Check if GPU is available
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TensorFlow")

print(tf.config.list_physical_devices('GPU'))

# Hyperparameters
patience = 4  # Stop training if accuracy doesn't improve
min_delta = 0.001  # Minimum tolerable difference in accuracy
filepath = "best_model_{epoch:02d}-{val_accuracy:.4f}.h5"

# Calculate class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_generator.classes),y=train_generator.classes)

# Convert class weights to dictionary format
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Build and compile the ResNet model
with tf.device('/GPU:0'):  # Ensure model is built on GPU
    resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    resnet_model.trainable = True

    model = Sequential([
        resnet_model,
        GlobalAveragePooling2D(),
        Flatten(),
        Dense(512, activation='relu'),  # Adding a dense layer
        Dropout(0.5),  # Adding dropout for regularization
        Dense(1, activation='sigmoid')
    ])
    model.summary()


In [ ]:
layers = [(layer.name, layer.trainable) for layer in model.layers]
df = pd.DataFrame(layers, columns=['Layer Name', 'Layer Trainable'])
pd.set_option('display.max_rows', None)
print(df)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall()])

In [ ]:
# callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=patience, min_delta=min_delta, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001, verbose=1)

HistoryModel = None

# Train the model
with tf.device('/GPU:0'):  # Ensure training is done on GPU
    history = model.fit(
        train_generator,
        epochs=50,  # Increase the number of epochs
        validation_data=validation_generator,
        callbacks=[early_stopping, model_checkpoint, reduce_lr],
        class_weight=class_weights_dict,
        verbose=1
    )
    HistoryModel = history

model.save('trained_resnet_model_real_fake_Optimized.h5')

In [ ]:
import keras
from matplotlib import pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
# Define paths to your data
test_data_dir = "Testing" 
img_height, img_width = 224, 224

# Loading your trained ResNet50 model
model = load_model("Best/best_model_21-0.9390.h5")

# Define data generators for testing
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=8, 
    class_mode='binary'  
)
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
# Print the test accuracy
print(f"Test Accuracy: {test_acc:.4f}")